----------
Questions to Martin (or to Martin's article):
----------

1. Before I could do convolutions and maxpoolings of 7 with stride 3 with inputs of 14x30, but now there is just 30. Should I try different filter sizes or is there one already predefined?

In [ ]:
#Here we go...
import dataWindows as data
from tensorflow.python.framework import ops
ops.reset_default_graph()
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt

To do here
------------
Fix the way the labels are being given since, they were working eg... to give result 5 (0 0 0 0 1 0 0 0 0 0)

In [14]:
#Directories of the data
data_dir = '/media/data/gomez/Padded_data_Sub_Jhmdb/Cut_each_3(Martins_experiment)/windows/windows_data1.mat'
label_dir = '/media/data/gomez/Padded_data_Sub_Jhmdb/Cut_each_3(Martins_experiment)/windows/windows_labls1.mat'
data_,labels_ = data.loadAll(data_dir,label_dir)

train_data = data_['trainD']
test_data_max = data_['testMax']
test_data_avg = data_['testAvg']
train_labels = labels_['trainL']
test_labels = labels_['testL']

print(test_labels.shape)

(89, 1)


In [3]:
'''print(train_data[0])
print(train_labels[0])'''
print('a passed test')
print(type(train_data))

a passed test
<class 'numpy.ndarray'>


In [4]:
#Parameters of training and viewing probabilities of the 
training_epochs = 10
drop_out_prob = 0.5
display_step = 10
iterations = 20000
learning_rate = 0.000001
tb_logs_path = '/home/gomez/Documents/Thesis_doc/tb_net_result_windows'

In [5]:
train_size,shpeData = train_data.shape
_,shpeLabl = train_labels.shape
print('shape of the data', shpeData, 'shape of the labels', shpeLabl, 'size of the train',train_size,_)
'''batch_size,shpeData = dataTest.shape
_,shpeLabl = labelsTest.shape
print('test',shpeData,shpeLabl,batch_size,_)'''

#Modify the data according to the given shape above
with tf.name_scope('input'):
    batch_size = tf.placeholder(tf.int32)
    x = tf.placeholder(tf.float32,[None,shpeData]) #[batch_size, shape_data_training_data]
    y_ = tf.placeholder(tf.float32,[None,shpeLabl]) #[batch_size]     

shape of the data 30 shape of the labels 1 size of the train 2795 2795


In [6]:
#Weight initialization

def weight_variable(shape, name, var_type): #Change the type in order to specify the variable to use
    with tf.name_scope('weight_' + name):
        initial = tf.random_uniform(shape)
        if(var_type == 'normal'):
            initial = tf.random_normal(shape, stddev=0.1)
        elif(var_type == 'trunc'): #As seen in the beginning
            initial = tf.truncated_normal(shape, stddev=0.1)
        w_variable = tf.Variable(initial)
        #tf.scalar_summary('_weight_' + name, w_variable)
    return w_variable

def bias_variable(shape, name):
    with tf.name_scope('bias_' + name):
        initial = tf.constant(0.1, shape=shape)
        b_variable = tf.Variable(initial)
        #tf.scalar_summary('_bias_' + name,b_variable)
    return b_variable

#Convolution and pooling (Martin: stride 30 for convolution, article: max_pooling of 7 with stride 7)

'''Change in the convolution from 30 to 3 (since size of input = 30) stride remains equal'''
def conv1d(x,w):
    with tf.name_scope('Conv1d'):
        convolution = tf.nn.conv1d(x,w,stride=3, padding='SAME')
        #tf.scalar_summary('_Conv1d_',convolution)
    return convolution

'''Change in the convolution from 7 to 3 (since size of input = 30)'''
def max_pool_3(x):    
    with tf.name_scope('max_pool_stride3'):
        max_pool7 = tf.nn.max_pool(x,ksize=[1,1,3,1],strides=[1,1,3,1], padding='SAME') #There is just max_pool in 2d and 3d so reshape and reshape
        #tf.scalar_summary('_max_pool_stride7_',max_pool7)
    return max_pool7

-----------
Remarks of this new version
-----------

w_conv1: changed to be a filter of size 30 to a filter of size 3 but same output to 32 
h_conv1_4d: Changed to have from a filter of 30 to 3, so the shpeDate/30 will be also modified to shpeDate/3

In [7]:
#First Layer

w_conv1 = weight_variable([3,1,32], 'conv1', 'trunc') #Kernel size of 3 and from 1 input to 32 neurons
b_conv1 = bias_variable([32], 'conv1') #32 biases going to different neurons

x_data = tf.reshape(x, [1,shpeData,1]) #Image converted in 3d

h_conv1 = tf.nn.relu(conv1d(x_data, w_conv1) + b_conv1)

print(h_conv1)

'''data: [3d] -> maxpooling(data): [4d] -> reconversion: [3d]'''

# h_conv1_4d:[batch_size,1 (1d data), 28 (840/30 = after convolution 30 (filter and stride) ), 32 (the output to the new network)]

'''From here, data becomes into a linnear value'''

h_conv1_4d = tf.reshape(h_conv1,[1,1,-1,32]) #Check at this change made here (Before I converted everything to linnear, now it remains same as input)

h_pool1 = max_pool_3(h_conv1_4d)

print(h_pool1)

#2 rshp_h_pool1:[batch_size, , 32 (output network)]

rshp_h_pool1 = tf.reshape(h_pool1,[1,-1,32]) #shape
print(rshp_h_pool1)

Tensor("Relu:0", shape=(1, 10, 32), dtype=float32)
Tensor("max_pool_stride3/MaxPool:0", shape=(1, 1, 4, 32), dtype=float32)
Tensor("Reshape_2:0", shape=(1, 4, 32), dtype=float32)


In [8]:
#Fully Connected layer 1

W_fc1 = weight_variable([4*32,64], 'fc1','trunc') #weights converted from the dimensions
b_fc1 = bias_variable([64], 'fc1')

h_pool2_flat = tf.reshape(rshp_h_pool1, [-1, 4*32])
print(h_pool2_flat)
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
print(h_fc1)

Tensor("Reshape_3:0", shape=(1, 128), dtype=float32)
Tensor("Relu_1:0", shape=(1, 64), dtype=float32)


In [9]:
#Fully Connected layer 2

W_fc2 = weight_variable([64,128], 'fc2', 'trunc') #weights converted to   --> Modified from [128,256] to [64,128]
b_fc2 = bias_variable([128], 'fc2') #Bias values going to the neurons

h_fc2 = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2) #Relu #2 -> Check if this was right -> Should be (before the h_pool2 was)
print(h_fc2)

Tensor("Relu_2:0", shape=(1, 128), dtype=float32)


In [10]:
# Dropout trick 

keep_prob = tf.placeholder("float")
h_fc2_drop = tf.nn.dropout(h_fc2,keep_prob)
print(h_fc2_drop)

# Output layer

W_fc3 = weight_variable([128,1], 'fc3', 'trunc') #Modified here
b_fc3 = bias_variable([1], 'fc3')

eps = tf.constant(0.00000001, shape=[1]) #Constant value added to prevent underflow (probability of having zero terms)
y_conv = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3) + eps
print(y_conv)

Tensor("dropout/mul_1:0", shape=(1, 128), dtype=float32)
Tensor("add_4:0", shape=(1, 1), dtype=float32)


In [11]:
#Train and evaluate
with tf.name_scope('cross_entropy'):
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.arg_max(y_conv,1),tf.arg_max(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

tf.scalar_summary('cost', cost)
tf.scalar_summary('accuracy', accuracy)

<tf.Tensor 'ScalarSummary_1:0' shape=() dtype=string>

In [12]:
#Merging all summaries and initializing all of the variables
summary_op = tf.merge_all_summaries()
init = tf.initialize_all_variables()

In [13]:
#Test

batch_x = data.next_batch(train_data,0,200)
batch_y = data.next_batch(train_labels,0,200)
with tf.Session() as sess:
    sess.run(init)
    [test_res1,test_res2] = sess.run([rshp_h_pool1, h_pool2_flat], feed_dict={x:batch_x, y_:batch_y, batch_size:batch_x.shape[0]})#, keep_prob:drop_out_prob})
    
print(test_res1.shape,test_res2.shape)
#print('use just to test size of each layer')

InvalidArgumentError: Input to reshape is a tensor with 6000 values, but the requested shape has 30
	 [[Node: Reshape = Reshape[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_input/Placeholder_1_0/_3, Reshape/shape)]]
	 [[Node: Reshape_2/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_35_Reshape_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op 'Reshape', defined at:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/gomez/Documents/virtual/lib/python3.4/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/gomez/Documents/virtual/lib/python3.4/site-packages/traitlets/config/application.py", line


Solution for the problem: Debugging
----------
Check the data initially after the max pooling, if the problem is not found then look from the beginning

--------------


In [ ]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.train.SummaryWriter(tb_logs_path,graph=tf.get_default_graph())    
    #Data used to plot the accuracy per each 20 steps
    x_iter = np.array([])
    y_acc = np.array([])
    max_acc_test = [0,0]
    #Data used in order to do the batching according to the size of the training_data
    # 1. Batch_size will be of 200 (from 2975 training sets)
    batch_size = 89
    # 2. Rounds will be 10000 to start.
    rounds = 10000
    # The new data
    for step in range(rounds):
        #The batches used to train the data
        batch_x = data.next_batch(train_data,step*batch_size,batch_size)
        batch_y = data.next_batch(train_labels,step*batch_size,batch_size)
        _, summary = sess.run([train_step,summary_op], feed_dict={x:batch_x, y_:batch_y, keep_prob: drop_out_prob})
        writer.add_summary(summary, step)
        '''This probably wont work because of the fixed numbers '''
        if step % display_step == 0: 
            #Calculates batch accuracy
            train_acc, loss = sess.run([accuracy, cost], feed_dict={x:batch_x, y_:batch_y, keep_prob: drop_out_prob})
            print(' Iteration: ' + str(step) + ' , loss: ' + '{:.6f}'.format(loss) + ' Accuracy: ' + '{:.6f}'.format(train_acc))
        if step % (display_step * 5) == 0:
            #Calculates accuracy in test
            test_acc = sess.run(accuracy, feed_dict={x:dataTest, y_:labelsTest, keep_prob:drop_out_prob})
            print("test_acc",test_acc)
            if(max_acc_test[1]<test_acc):
                max_acc_test[:] = [step,test_acc]
            y_acc = np.append(y_acc,test_acc)
            x_iter= np.append(x_iter,step)
            
    print('Optimization Finished')
    test_acc = sess.run(accuracy, feed_dict={x:dataTest, y_:labelsTest, keep_prob:drop_out_prob})
    print('And final test accuracy is:',test_acc)   

In [ ]:
print('maximum accuracy obtained is', max_acc_test[1], 'in iteration', max_acc_test[0])
plt.plot(x_iter,y_acc)
plt.show()